In [15]:
from ipyfilechooser import FileChooser
from IPython.display import display
import ipywidgets as widgets
import os
import clamp_ephys
import numpy as np
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [16]:
''' ################### SET/CHECK THESE PARAMETERS BEFORE RUNNING ################## '''
lowpass_freq = 500  # Hz
stim_time = 500     # ms
post_stim = 250     # ms, amount of time after stimulus to look for max value
tp_start = 5        # ms, time of start of test pulse
vm_jump = 10        # mV, test pulse voltage jump
pre_tp = 3          # ms, amount of time before test pulse start to get baseline
unit_scaler = -12   # unitless, scaler to get back to A, from pA
amp_factor = 1      # scaler for making plots in pA
fs = 25             # kHz, the sampling frequency
timepoint = 'p2'
home_dir = os.path.expanduser('~')
data_path = os.path.join(home_dir, 'urban', 'clamp_ephys', 'test_data')

In [17]:
data_choose = FileChooser()
data_choose.default_path = data_path
data_choose.use_dir_icons = True
data_choose.title = '<b>Select Data</b>'

data_notes_choose = FileChooser()
data_notes_choose.default_path = data_path
data_notes_choose.use_dir_icons = True
data_notes_choose.title = '<b>Select Data Notes</b>'

slider = widgets.IntSlider(value=0, min=0, max=10, step=1, continuous_update=False)

open_cell = widgets.Button(description='Open Cell', tooltip='open selected cell file with data notes')
textout = widgets.Output()
plotout = widgets.Output()


In [22]:
def open_cell_click(button):
    file = data_choose.selected_filename
    file_path = data_choose.selected
    data_notes = data_notes_choose.selected
    cell = clamp_ephys.workflows.cell(file_path, fs=fs, path_to_data_notes=data_notes, timepoint=timepoint, amp_factor=amp_factor)
    cell.filter_traces(lowpass_freq)
    trace = cell.traces_filtered[slider.value]
    time = np.arange(0, len(trace) / cell.fs, 1 / cell.fs)
    ntraces = len(cell.traces_filtered.columns)
    slider.max = ntraces - 1
    
    textout.clear_output()
    plotout.clear_output()

    with textout:
        print(f'{file} opened')

    with plotout:
        p = figure(plot_width=800, plot_height=400)
        sweep = p.line(time, trace)
        show(p, notebook_handle=True)

    def slider_change(change):
        with plotout:
            sweep.data_source.data['y'] = cell.traces_filtered[slider.value]
            push_notebook()
            
    slider.observe(slider_change, names='value')
    
open_cell.on_click(open_cell_click)

In [23]:
display(data_choose, data_notes_choose, open_cell, textout, slider, plotout)


FileChooser(path='/home/nate/urban/clamp_ephys/test_data', filename='', show_hidden='False')

FileChooser(path='/home/nate/urban/clamp_ephys/test_data', filename='', show_hidden='False')

Button(description='Open Cell', style=ButtonStyle(), tooltip='open selected cell file with data notes')

Output(outputs=({'name': 'stdout', 'text': 'JH200303_c1_light100.ibw opened\n', 'output_type': 'stream'},))

IntSlider(value=30, continuous_update=False, max=30)

Output(outputs=({'output_type': 'display_data', 'data': {'text/html': '\n\n\n\n\n\n  <div class="bk-root" id="…